# Visualizing the log data:

#### Importing packages

In [1]:
from glob import glob
import warnings 
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from polyphys.visualize import tuner
from polyphys.visualize.plotter import PAUL_TOL_VIBRANT as PTV
from polyphys.visualize import plotter
from polyphys.manage.parser import SumRuleCyl, TransFociCyl

# Slurm runtimes

## Simple style

In [6]:
def merge_rows(group):
    total_runtime = group['total_runtime_sec'].sum() # Sum the runtimes
    n_cores = group['ncores'].iloc[0] # Take the first 'n_cores' value
    base_sim = group.name # 'base_sim' is the group name
    return pd.DataFrame({'whole_folder': [base_sim], 
                         'total_runtime_sec': [total_runtime], 
                         'ncores': [n_cores]})

# Function to apply SumRuleCyl and extract properties
def extract_properties(whole_name, lineage, geometry, group, topology):
    cyl = SumRuleCyl(whole_name, lineage, geometry, group, topology, ispath=False)
    properties = {}
    for attr in cyl.attributes:
        properties[attr] = getattr(cyl, attr)
    for attr in cyl.genealogy:
        properties[attr] = getattr(cyl, attr)
    return properties

lineage = 'whole'
geometry = 'cylindrical'
group = 'bug'
topology = 'linear'
project_name = 'SumRuleCyl-N2000D240.0'

database = '/Users/amirhsi_mini/research_data/runtimes/N*'
df_paths = glob(database)
runtimes = pd.concat([pd.read_csv(df_path) for df_path in df_paths])
col_new_names = dict(zip(runtimes.columns,['folder','total_runtime_sec','ncores']))
runtimes.rename(columns=col_new_names, inplace=True)
runtimes['whole_folder'] = runtimes['folder'].str.extract(r'(N.*ens[1-8])(?:_res|_incomplete)?')
# Group by the base simulation name
grouped = runtimes.groupby('whole_folder')
runtimes = grouped.apply(merge_rows).reset_index(drop=True)
# Assuming 'df' is your DataFrame and 'total_runtime_sec' is the column
runtimes['total_runtime_hours'] = runtimes['total_runtime_sec'] / 3600
# Apply to the dataframe
runtimes_properties = runtimes['whole_folder'].apply(extract_properties, args=(lineage, geometry, group, topology))
# Convert the series of dictionaries to a dataframe
runtimes_properties = pd.DataFrame(runtimes_properties.tolist())
# Merge this dataframe with the original dataframe
runtimes_merged = pd.concat([runtimes, runtimes_properties], axis=1)
runtimes_merged['natoms'] = runtimes_merged['nmon'] + runtimes_merged['ncrowd'] 


In [16]:
runtimes_merged[['whole','space','natoms','total_runtime_hours', 'ncores']].sort_values(by=['ncores','natoms']).to_csv('test.csv')

In [ ]:
fontsize = 14
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 1.5
height = 6
aspect = 1.618
ylabel_pad = 50
rc_params= {
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,        
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'bbox_to_anchor': (0.33, 0.33),
    'markerscale': 1.5
}

sns.set_theme(
    context=plot_context,
    style='ticks',
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
stat_grid = sns.relplot(
    data=runtimes_merged,
    x='natoms',
    y='total_runtime_hours',
    col='space',
    col_wrap=3,
    hue='ncores',
    kind='line',
    marker='s',
    height=height,
    aspect=aspect,
    palette=PTV,
    facet_kws=facet_kws
)

stat_grid.set_xlabels(r"$n_{atoms}$")
stat_grid.set_ylabels(r"Total wall time (hrs)")
sns.move_legend(stat_grid, "upper left", title=r"$n_{cores}$", **move_legend_kws)
output = "-".join(["wallTimeSlurm", project_name]) + "." + ext
stat_grid.savefig(save_to + output, bbox_inches='tight')
plt.close()

## Advanced style with neighbor settings

In [11]:
# Function to apply SumRuleCyl and extract properties
def extract_properties(whole_name, lineage, geometry, group, topology):
    cyl = SumRuleCyl(whole_name, lineage, geometry, group, topology, ispath=False)
    properties = {}
    for attr in cyl.attributes:
        properties[attr] = getattr(cyl, attr)
    for attr in cyl.genealogy:
        properties[attr] = getattr(cyl, attr)
    return properties

lineage = 'whole'
geometry = 'cylindrical'
group = 'bug'
topology = 'linear'
project_name = 'SumRuleCyl-N2000D120.0_240.0'

database = '/Users/amirhsi/research_data/runtimes/N*'
df_paths = glob(database)
runtimes = pd.concat([pd.read_csv(df_path) for df_path in df_paths])
col_new_names = dict(zip(runtimes.columns,['folder','total_runtime_sec','ncores', 'avg_step_per_sec', 'bin_scheme', 'rskin']))
runtimes.rename(columns=col_new_names, inplace=True)
runtimes['whole_folder'] = runtimes['folder'].str.extract(r'(N.*ens[1-8])(?:_res|_incomplete|_cont)?')
# Group by the base simulation name
print(runtimes.columns)
grouped = runtimes.groupby(['whole_folder', 'bin_scheme', 'rskin']).agg({'ncores': 'last', 'total_runtime_sec': np.sum, 'avg_step_per_sec': np.mean})
runtimes = grouped.reset_index()
# Assuming 'df' is your DataFrame and 'total_runtime_sec' is the column
runtimes['total_runtime_hours'] = runtimes['total_runtime_sec'] / 3600
# Apply to the dataframe
runtimes_properties = runtimes['whole_folder'].apply(extract_properties, args=(lineage, geometry, group, topology))
# Convert the series of dictionaries to a dataframe
runtimes_properties = pd.DataFrame(runtimes_properties.tolist())
# Merge this dataframe with the original dataframe
runtimes_merged = pd.concat([runtimes, runtimes_properties], axis=1)
runtimes_merged['natoms'] = runtimes_merged['nmon'] + runtimes_merged['ncrowd'] 
runtimes_merged['runtime_3.01e8'] = 602 * runtimes['total_runtime_hours']
print(runtimes_merged.isna().sum())
runtimes_merged.dropna(inplace=True)

Index(['folder', 'total_runtime_sec', 'ncores', 'avg_step_per_sec',
       'bin_scheme', 'rskin', 'whole_folder'],
      dtype='object')
whole_folder           0
bin_scheme             0
rskin                  0
ncores                 0
total_runtime_sec      0
avg_step_per_sec       3
total_runtime_hours    0
nmon                   0
epsilon                0
dcyl                   0
lcyl                   0
dcrowd                 0
ncrowd                 0
dt                     0
bdump                  0
adump                  0
ensemble_id            0
dmon                   0
mmon                   0
eps_others             0
mcrowd                 0
dwall                  0
phi_m_bulk             0
rho_m_bulk             0
phi_c_bulk             0
rho_c_bulk             0
lineage_name           0
whole                  0
ensemble_long          0
ensemble               0
space                  0
natoms                 0
runtime_3.01e8         0
dtype: int64


In [10]:
fontsize = 14
save_to = '../../test_plots/'
ext = 'pdf'
plot_context = 'talk'
font_scale = 1.5
height = 6
aspect = 1.618
ylabel_pad = 50
rc_params= {
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,        
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'bbox_to_anchor': (0.33, 0.33),
    'markerscale': 1.5
}
#y_attr = 'avg_step_per_sec' #'total_runtime_hours'
#y_attr = 'total_runtime_hours'
y_attr = 'runtime_3.01e8'
sns.set_theme(
    context=plot_context,
    style='ticks',
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
stat_grid = sns.relplot(
    data=runtimes_merged,
    x='natoms',
    y=y_attr,
    col='space',
    col_wrap=3,
    style='ncores',
    dashes=False,
    markers=True,
    hue='ncores',
    kind='line',
    height=height,
    aspect=aspect,
    palette=PTV,
    facet_kws=facet_kws
)

stat_grid.set_xlabels(r"$n_{atoms}$")
stat_grid.set_ylabels(r"Total wall time (hrs)")
#stat_grid.set_ylabels(r"Avg. time step per second")
#sns.move_legend(stat_grid, "upper left", title=r"$n_{cores}$", **move_legend_kws)
output = "-".join(["wallTimeSlurm", project_name, y_attr]) + "." + ext
stat_grid.savefig(save_to + output, bbox_inches='tight')
plt.close()

# Lammps logs

### Importing datasets

In [ ]:
database = '../../Datasets/'
project = 'SumRuleCyl'
df_path = database + project + "-allInOne/"
#thermo = pd.read_parquet(
#    df_path + "allInOne-" + project + "-thermo.parquet.brotli")
run_stat = pd.read_csv(df_path + "allInOne-" + project + "-runStat-misssing_D60ac4_6.csv")
wall_time = pd.read_csv(df_path + "allInOne-" + project + "-wallTimeStat-misssing_D60ac4_6.csv")

## Ensemble average over run and wall time stats

In [ ]:
run_stat.loop_timesteps.value_counts()

In [ ]:
space_chosen = ['N1000D15.0ac1.0', 'N1000D15.0ac2.0', 'N2000D20.0ac1.0',
                'N2000D20.0ac2.0', 'N2000D25.0ac1.0', 'N2000D25.0ac2.0',
                'N2000D25.0ac4.0']
loop_timesteps_chosen = [5000000, 10000000]
gby_cols = ['n_cores', 'n_atoms', 'ensemble_long', 'space']
props = ['loop_idx', 'loop_time', 'loop_timesteps', 'tau_day', 'lineage_name', 'whole', 'segment', 'ensemble']
props_func = ['count', 'sum',  'sum',  'mean', 'count', 'count', 'count', 'count']
prop_dics = dict(zip(props, props_func))
prop_dics['timestep_sec'] = ['min', 'max', 'sum']
#run_stat_filt = run_stat.loc[(run_stat['space'].isin(space_chosen)) & (run_stat['loop_timesteps']==loop_timesteps_chosen)]
run_stat_filt = run_stat.loc[run_stat['loop_timesteps'].isin(loop_timesteps_chosen)]
run_stat_filt.reset_index(inplace=True,drop=True)
#run_stat_filt.reset_index(inplace=True, drop=True)
run_stat_filt = run_stat_filt.groupby(gby_cols).agg(prop_dics)
#run_stat_filt# = run_stat_filt[props]
run_stat_filt.reset_index(inplace=True)
run_stat_filt.columns = ['_'.join(col).rstrip('_') for col in run_stat_filt.columns.values]
run_stat_filt['timestep_sec_mean'] = run_stat_filt['timestep_sec_sum'] / run_stat_filt['loop_idx_count'] 

In [ ]:
#run_stat_filt.to_csv("../../Datasets/SumRuleCyl-allInOne/allInOne-SumRuleCyl-runStat-N1000_2000D20_25-min_mean_max.csv",index=False)
run_stat_filt.to_csv("../../Datasets/SumRuleCyl-allInOne/allInOne-SumRuleCyl-runStat-min_mean_max.csv",index=False)

### Visualizing the run stats: 

In [ ]:
## This is for SumRUleCyl project:
# droping prblematic (broken, restarted, faked) run stats and keep complete
# ones:
run_stat_comp = run_stat[run_stat.lineage_name.str.contains('ens[1-8]$|j0[12]$')]
run_stat_comp.reset_index(inplace=True,drop=True)
run_stat_comp
# keep data from the production phase in whic the loop_timesteps is equal to
# 5000000 ro larger than that:
#run_stat_prod = run_stat_comp[run_stat_comp.loop_timesteps >= 5000000]
sel_cols = ['Pair_total_pct', 'Bond_total_pct', 'Neigh_total_pct',
            'Comm_total_pct', 'Output_total_pct', 'Modify_total_pct',
            'Other_total_pct']
agg_funcs = {key: np.mean for key in sel_cols}
run_stats_per_cor_per_atoms = run_stat_prod.groupby(['n_cores','n_atoms'])[sel_cols].agg(agg_funcs)
run_stats_per_cor_per_atoms.reset_index(inplace=True)
run_stats_per_cor_per_atoms_per_cat = pd.melt(
    run_stats_per_cor_per_atoms,
    id_vars=['n_cores','n_atoms'],
    value_vars= sel_cols,
    var_name='category',
    value_name='pct_of_total_time'
)

In [ ]:
sel_cols = ['Pair_total_pct', 'Bond_total_pct', 'Neigh_total_pct',
            'Comm_total_pct', 'Output_total_pct', 'Modify_total_pct',
            'Other_total_pct']
agg_funcs = {key: np.mean for key in sel_cols}
run_stats_per_cor_per_atoms = run_stat.groupby(['n_cores','n_atoms'])[sel_cols].agg(agg_funcs)
run_stats_per_cor_per_atoms.reset_index(inplace=True)
run_stats_per_cor_per_atoms_per_cat = pd.melt(
    run_stats_per_cor_per_atoms,
    id_vars=['n_cores','n_atoms'],
    value_vars= sel_cols,
    var_name='category',
    value_name='pct_of_total_time'
)

In [ ]:
fontsize = 14
save_to = "./"
ext = 'pdf'
new_labels = ['Category', 'Pair', 'Bond', 'Neigh', 'Comm', 'Output', 'Modify', 'Other', '# of cores', '1', '4', '8', '16']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,4.5))
sns.set_context(
    font_scale=3,
    rc={
        'font.family': "Times New Roman",
        'mathtext.default': 'regular',
        "text.usetex": True,
        "font.size": fontsize
    }
)
ax_sns = sns.lineplot(
    x='n_atoms',
    y='pct_of_total_time',
    hue='category',
    style='n_cores',
    legend='full',
    markers=True,
    data=run_stats_per_cor_per_atoms_per_cat,
    ax=ax
)
ax_sns.set_xlabel(r"# of particles")
ax_sns.set_ylabel(r"% of the total time")
ax_sns.set(xscale='log')
#handles, labels = ax_sns.get_legend_handles_labels()
#ax_sns.legend(handles = handles, labels=new_labels, frameon=False)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
output = "-".join(["runStat", project]) + "." + ext
fig.savefig(save_to + output, bbox_inches='tight')

In [ ]:
sel_cols = [ 'n_cores', 'n_atoms','timestep_sec','dangerous_builds']
agg_funcs = {}
agg_funcs['timestep_sec'] = np.mean
agg_funcs['dangerous_builds'] = np.sum

In [ ]:
step_per_sec_stats = run_stat.groupby(['n_cores','n_atoms'])[sel_cols].agg(agg_funcs)
step_per_sec_stats.reset_index(inplace=True)

In [ ]:
step_per_sec_stats['timestep_sec_core'] = \
    step_per_sec_stats['timestep_sec'] / step_per_sec_stats['n_cores']

In [ ]:
fontsize = 14
save_to = "./"
ext = 'pdf'
#new_labels = ['Category', 'Pair', 'Bond', 'Neigh', 'Comm', 'Output', 'Modify', 'Other', '# of cores', '1', '4', '8', '16']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 6
aspect = 1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,        
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1.5
}

sns.set_theme(
    context=plot_context,
    style='ticks',
  #  palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
stat_grid = sns.relplot(
    x='n_atoms',
    y='timestep_sec_core',
    col='n_cores',
    col_wrap=2,
  #  marker='s',
    data=step_per_sec_stats,
    legend='full',
    kind='line',
    height=height,
    aspect=aspect,
    facet_kws=facet_kws,
)
col_attr = 'n_cores'
attr_labels = {
    'n_cores': r"$n_{{core}}$"
}
stat_grid.set_xlabels(r"$n_{atoms}$")
stat_grid.set_ylabels(r"${steps}/{sec.n_{core}}$")
stat_grid.set_titles(attr_labels[col_attr] + r"$={col_name}$")
stat_grid.tight_layout(w_pad=0)
#sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
#output = "-".join(["runStat", project_name]) + "." + ext
#fig.savefig(save_to + output, bbox_inches='tight')

### Visualizing wall-time stats

In [ ]:
wall_time.columns

In [ ]:
wall_time_per_whole = wall_time.groupby(['whole'])['ensemble','whole','n_cores', 'n_atoms','wall_time_hr'].agg({'ensemble': 'last','whole':'last','n_cores':'last', 'n_atoms':'last', 'wall_time_hr': np.sum})
wall_time_per_whole.reset_index(inplace=True,drop=True)
wall_time_per_ensemble = wall_time_per_whole.groupby(['ensemble'])['ensemble','n_cores','n_atoms','wall_time_hr'].agg({'ensemble': 'last','n_cores':'last', 'n_atoms':'last', 'wall_time_hr': np.mean})
wall_time_per_ensemble.reset_index(inplace=True,drop=True)
wall_time_per_ensemble.sort_values('n_cores')
wall_time_per_ensemble_sorted = wall_time_per_ensemble.sort_values(by=['n_cores','n_atoms'])
wall_time_per_ensemble_sorted

### Filtered data based on n_cores

In [ ]:
n_core = 32
filtered = wall_time_per_ensemble_sorted.loc[wall_time_per_ensemble_sorted['n_cores']==n_core,:]

fontsize = 12
save_to = "./"
ext = 'pdf'
#new_labels = ['Category', 'Pair', 'Bond', 'Neigh', 'Comm', 'Output', 'Modify', 'Other', '# of cores', '1', '4', '8', '16']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
sns.set_theme(
    context='paper',
    style='ticks',
    font='Times New Roman',
    font_scale=2,
    rc={
        'mathtext.default': 'regular',
        "text.usetex": True,
        "font.size": fontsize,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        'axes.grid.which': 'both'
    }
)
ax_sns = sns.regplot(
    x='n_atoms',
    y='wall_time_hr',
    marker='s',
    data=filtered,
    ax=ax
)

ax_sns.set_xlabel("Number of particles")
ax_sns.set_ylabel("Wall time (hours)")
#sns.move_legend(ax_sns, title="# of cores", loc= "upper left", bbox_to_anchor=(1, 1))
output = "-".join(["wallTimeStat", project, f'nCores{n_core}']) + "." + ext
fig.savefig(save_to + output, bbox_inches='tight')
#plt.close()

### Wall stats:

In [ ]:
fontsize = 12
save_to = "./"
ext = 'pdf'
#new_labels = ['Category', 'Pair', 'Bond', 'Neigh', 'Comm', 'Output', 'Modify', 'Other', '# of cores', '1', '4', '8', '16']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,9))
sns.set_theme(
    context='paper',
    style='ticks',
    font='Times New Roman',
    font_scale=2,
    rc={
        'mathtext.default': 'regular',
        "text.usetex": True,
        "font.size": fontsize,
        'axes.grid': True,
        'axes.grid.axis': 'both',
        'axes.grid.which': 'both'
    }
)
ax_sns = sns.lineplot(
    x='n_atoms',
    y='wall_time_hr',
    style='n_cores',
    hue='n_cores',
    marker='s',
    data=wall_time_per_ensemble_sorted,
    ax=ax
)

ax_sns.set_xlabel("Number of particles")
ax_sns.set_ylabel("Wall time (hours)")
sns.move_legend(ax_sns, title="Number of cores", loc= "upper left", bbox_to_anchor=(1, 1))
output = "-".join(["wallTimeStat", project]) + "." + ext
fig.savefig(save_to + output, bbox_inches='tight')
#plt.close()

### Visualizing thermodynamic data

In [ ]:
thermo.columns

In [ ]:
#sel_cols = ['Step', 'TotEng', 'KinEng', 'Temp', 'PotEng', 'E_bond', 'E_angle',
#       'E_dihed', 'E_impro', 'E_vdwl', 'E_coul', 'E_long', 'Press',
#        'whole', 'ensemble_long', 'ensemble', 'space',
#        'dcyl', 'dmon_large', 'nmon_large',
#        'nmon_small', 'dcrowd', 'dt', 'bdump',
#       'adump', 'ensemble_id',  'phi_c_bulk'] # multi
thermo_style = 'one' # 'multi'
sel_cols = {
    'SumRuleCyl': {
        'one': [
            'Step', 'Temp', 'E_pair', 'E_mol', 'TotEng', 'Press',
            'lineage_name', 'whole', 'ensemble_long', 'ensemble',
            'space', 'nmon', 'epsilon', 'dcyl', 'lcyl', 'dcrowd',
            'ncrowd', 'dt', 'bdump', 'adump', 'ensemble_id',
            'phi_m_bulk', 'rho_m_bulk', 'phi_c_bulk', 'rho_c_bulk'
        ]
    },
    'HnsCub': {
        'one': ['Step',
            'KinEng', 'PotEng', 'E_tail', 'Ecouple', 'Econserve', 'E_vdwl',
            'E_bond', 'E_angle', 'E_mol', 'Temp', 'Press', 'nhns', 'dcrowd',
            'ensemble_id', 'phi_c_bulk', 'lineage_name', 'whole',
            'ensemble_long', 'space', 'E_pair', 'TotEng'
        ]
    }
}
thermo_equil = thermo.loc[:,sel_cols[project][thermo_style]]

In [ ]:
round_to = 0.025
rounding_func = lambda x, round_to: np.round(np.rint((x / round_to)) * round_to, 3)
thermo_equil['phi_c_bulk_round'] = thermo_equil['phi_c_bulk'].apply(
    rounding_func, args=[round_to]
)

In [ ]:
sns.relplot(
    x="Step",
    y="E_bond",
    col='phi_c_bulk_round',
    col_wrap=3,
    hue='ensemble_id',
    ci=None,
    alpha=0.5,
    legend="full",
    kind="line",
    facet_kws={"sharey": False},
    data=thermo_equil
)

In [ ]:
sns.relplot(
    x="Step",
    y="Press",
    col='phi_c_bulk_round',
    col_wrap=3,
    hue='ensemble_id',
    ci=None,
    alpha=0.5,
    legend="full",
    kind="line",
    facet_kws={"sharey": False},
    data=thermos_equil
)

### Filter themrmo by time step

In [ ]:
thermp_freq = 10000
equilibration_total_steps = 1000000
sampling_first_step = equilibration_total_steps + thermp_freq
#thermos_equil = thermos.loc[thermos["Step"]>=sampling_first_step,:]
thermos_equil.reset_index(inplace=True, drop=True)

## Not finished: 

In [ ]:
colors = ['royalblue', 'firebrick']
lineage = 'whole'
geometry = 'biaxial'

if len(thermos) % 2 == 0:
    n_cols = len(thermos)//2
else:
    n_cols = len(thermos)//2 + 1
plt.rcParams.update({
    "text.usetex": True, 
})
mpl.rcParams['font.family'] = "Times New Roman"
mpl.rcParams['mathtext.default'] = "regular"
plt.rcParams['font.size'] = 14
fig, axes = plt.subplots(2, n_cols, sharex=False, figsize=(9,12))
xdata ='step'
ydata= 'press'
thermp_freq = 10000
equilibration_steps = 1000000
sampling_first_step = equilibration_steps + thermp_freq
sampling_first_row = sampling_first_step // thermp_freq
for idx, (ax, color, (data_path, data_org)) in enumerate(zip(axes.flat, colors, thermos.items())):
    data_info = TransFoci(
            data_path,
            geometry=geometry,
            group='bug',
            lineage=lineage
        )
    data = data_org.copy()
    data.drop_duplicates(inplace=True)
    data.reset_index(inplace=True, drop=True)
    #data = data.iloc[sampling_first_row:,:]
    y_mean = data.loc[sampling_first_row:,ydata].mean()
    ax.set_title(fr"$\phi_c^{{bulk}}={np.round(data_info.phi_c_bulk,3)}$")
    ax.axhline(
        y=y_mean, alpha=0.8, ls='--',
        c=color, 
        label=fr"$\bar{{P}}={np.round(y_mean,3)}$"
        )
    ax.plot(
        data.loc[sampling_first_row:,xdata],
        data.loc[sampling_first_row:,ydata], color=color, alpha=0.7,
            label=r"$P(t)$"
    )    
    ax.set_ylabel(r"Pressure, $P(t)$")
    #ax.set_xlabel(r"Time, ${t}/{\Delta t}$")
    ax.set_xlabel(r"Time Step")
    ax.legend()
fname = ydata+"-time-"+data_info.space+"."+'pdf'
fig.tight_layout()
plt.savefig(fname, bbox_inches='tight')
#plt.close()

In [ ]:
_, axes = plt.subplots(nrows=2, ncols=1, figsize=(16, 9))
for idx, (ax, color, (data_path, data_org)) in enumerate(zip(axes.flat, colors, thermos.items())):
    #transition_time = 5500
    data = data_org.copy()
    data.drop_duplicates(inplace=True)
    data.reset_index(inplace=True)
    #Use of the following options:
    #data = gyr_t[:transition_time] # before transition at t~5500
    data = data['eVdwl'].to_numpy() # after transition at t~5500
    data = data[101:]
    #data = gyr_t # whole date
    result = analyzer.error_calc_block(data, './block_analysis') 
    #ax.plot(result['si'])
    ax.grid(True, which="both")
    ax.errorbar(result['ntransfroms'], result['si'], yerr=result['si_err'], fmt='--o')
    ax.set_xlabel(r"Number of transformation, $n_{block}$")
    ax.set_ylabel(r"Statistical inefficiency, $s(n_{block})$")

In [ ]:
colors = ['royalblue', 'firebrick']
lineage = 'whole'
geometry = 'biaxial'

if len(thermos) % 2 == 0:
    n_cols = len(thermos)//2
else:
    n_cols = len(thermos)//2 + 1
plt.rcParams.update({
    "text.usetex": True, 
})
mpl.rcParams['font.family'] = "Times New Roman"
mpl.rcParams['mathtext.default'] = "regular"
plt.rcParams['font.size'] = 14
fig, axes = plt.subplots(2, n_cols, sharex=False, figsize=(9,12))
xdata ='step'
ydata= 'press'
thermp_freq = 10000
equilibration_steps = 1000000
sampling_first_step = equilibration_steps + thermp_freq
sampling_first_row = sampling_first_step // thermp_freq
for idx, (ax, color, (data_path, data_org)) in enumerate(zip(axes.flat, colors, thermos.items())):
    data_info = TransFoci(
            data_path,
            geometry=geometry,
            group='bug',
            lineage=lineage
        )
    data = data_org.copy()
    data.drop_duplicates(inplace=True)
    data.reset_index(inplace=True, drop=True)
    #data = data.iloc[sampling_first_row:,:]
    y_mean = data.loc[sampling_first_row:,ydata].mean()
    ax.set_title(fr"$\phi_c^{{bulk}}={np.round(data_info.phi_c_bulk,3)}$")
    correlations(data.loc[sampling_first_row:,ydata], lags=40)
    ax.set_ylabel(r"Pressure, $P(t)$")
    #ax.set_xlabel(r"Time, ${t}/{\Delta t}$")
    ax.set_xlabel(r"Time Step")
    ax.legend()
fname = ydata+"-time-"+data_info.space+"."+'pdf'
fig.tight_layout()
plt.savefig(fname, bbox_inches='tight')
#plt.close()